In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U --q transformers datasets accelerate sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# Import library

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import datasets
from sklearn.metrics import *

# Load data

In [4]:
sufix_path = "/content/drive/MyDrive/NLU_NCKH/notebook/Data/Hotel Preprocessed"

df_train = pd.read_csv(f"{sufix_path}/Train.csv")
df_dev = pd.read_csv(f"{sufix_path}/Dev.csv")
df_test = pd.read_csv(f"{sufix_path}/Test.csv")

print("Train: ", df_train.shape)
print("Dev: ",  df_dev.shape)
print("Test: ", df_test.shape )

Train:  (7180, 35)
Dev:  (795, 35)
Test:  (2030, 35)


# Prepare data for model `vinai/phobert-base`

In [5]:
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, aspects, tokenizer, MAX_LEN, type_model="train") -> None:
    self.df = df
    self.aspects = aspects
    self.tokenizer = tokenizer
    self.MAX_LEN = MAX_LEN
    self.type_model = type_model
  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    review = row['review']
    aspects = row[self.aspects]

    inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.MAX_LEN,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]

    if self.type_model == "test":
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

    label = self.label_one_hot(aspects)
    label = label.reshape(-1)
    return {
        'input_ids': torch.tensor(ids, dtype=torch.long),
        'attention_mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        'labels': torch.tensor(label, dtype=torch.float)
    }


  def label_one_hot(self, label):
    encoder = OneHotEncoder(categories=[[0, 1, 2, 3]], dtype='uint8', sparse=False)
    one_hot_outputs = encoder.fit_transform(label.values.reshape(-1, 1))
    return one_hot_outputs


MODEL_NAME = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Hyperparameters
MAX_LEN = max(len(tokenizer.encode(review)) for review in df_train['review'])
TRAIN_BATCH_SIZE = 40
VALID_BATCH_SIZE = 20
EPOCHS = 20
LEARNING_RATE = 2e-05

aspects = df_train.drop(columns='review').columns.tolist()

tokenizer_train = CustomDataset(df_train, aspects, tokenizer, MAX_LEN)
tokenizer_test = CustomDataset(df_test, aspects, tokenizer, MAX_LEN, "test")
tokenizer_dev = CustomDataset(df_dev, aspects, tokenizer, MAX_LEN)

print("Encoded: ",tokenizer_dev.__getitem__(1)['input_ids'].size())
print("Decoded: ",tokenizer.decode(tokenizer_dev.__getitem__(1)['input_ids'], skip_special_tokens=True))
print("Label: ",tokenizer_dev.__getitem__(1)['labels'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Encoded:  torch.Size([150])
Decoded:  nếu số_lượng món ăn nhiều hơn thì tốt biết_mấy
Label:  tensor([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])


# Prepare DataLoader

In [6]:
from torch.utils.data import DataLoader

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(tokenizer_train, **train_params)
validating_loader = DataLoader(tokenizer_dev, **test_params)
testing_loader = DataLoader(tokenizer_test, **test_params)

for k, v in next(iter(training_loader)).items():
  print(f"{k}: {v.size()}")

input_ids: torch.Size([40, 150])
attention_mask: torch.Size([40, 150])
token_type_ids: torch.Size([40, 150])
labels: torch.Size([40, 136])


# Traing loop

## Define model for training loop

In [7]:
import torch
from torch import nn

class phoBERTClass(torch.nn.Module):
    def __init__(self, MODEL_NAME, num_labels):
        super(phoBERTClass, self).__init__()
        self.pretrained_bert = AutoModel.from_pretrained(MODEL_NAME, output_hidden_states=True)
        self.dropout = nn.Dropout(0.2)
        self.num_labels = num_labels
        self.classifier = nn.ModuleList(
            [nn.Linear(self.pretrained_bert.config.hidden_size * 4, 4)
              for _ in range(self.num_labels)
            ]
          )

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.pretrained_bert(input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                      )
        hidden_states = outputs.hidden_states

        # Concatenate last 4 hidden states
        pooled_output = torch.cat(tuple([hidden_states[i] for i in range(-4, 0)]), dim=-1)
        x = self.dropout(pooled_output[:, 0, :])

        outputs = [nn.functional.softmax(dense_layer(x), dim=-1) for dense_layer in self.classifier]

        return torch.cat(outputs, dim=-1)

## Define train and evaluate

In [8]:
from tqdm.auto import tqdm
from sklearn import metrics
import time

def train_epoch(model, optimizer, loss_fn, train_loader, device, progress_bar, epoch):
    model.train()
    losses = []
    for index, data in tqdm(enumerate(train_loader), desc=f'Training {epoch}', total=len(train_loader)):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['labels'].to(device, dtype = torch.float)

          outputs = model(ids, mask, token_type_ids)

          optimizer.zero_grad()
          loss = loss_fn(outputs, targets)
          if index%100 == 0 or index == (len(training_loader) - 1):
              print(
                "| Epoch {:3d} | Index: {:4d} | Train Loss {:8.3f}"
                .format(
                    epoch, index, loss.item()
                  )
              )
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          progress_bar.update(1)

          losses.append(loss.item())

    return sum(losses)/len(losses)

def evaluate_epoch(model, loss_fn, valid_loader, device):
      model.eval()
      fin_targets=[]
      fin_outputs=[]
      losses = []
      with torch.no_grad():
          for _, data in tqdm(enumerate(valid_loader, 0), desc="Evaluating", total=len(valid_loader)):
              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              targets = data['labels'].to(device, dtype = torch.float)

              outputs = model(ids, mask, token_type_ids)

              loss = loss_fn(outputs, targets)
              losses.append(loss)
              fin_targets.extend(targets.cpu().detach().numpy().tolist())
              fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      return {
          "loss": sum(losses)/len(losses),
        }

def train(model, model_name, save_model, optimizer, loss_fn, train_loader,
          valid_loader, num_epochs, device, num_training_steps):
      progress_bar = tqdm(range(num_training_steps), desc="Training")
      train_losses = []
      metrics_eval = []
      best_loss_eval = 100
      times = []

      for epoch in range(1, num_epochs+1):
          epoch_start_time = time.time()
          # Training
          train_loss = train_epoch(model, optimizer, loss_fn, train_loader, device, progress_bar, epoch)
          train_losses.append(train_loss)

          # Evaluation
          eval_metric = evaluate_epoch(model, loss_fn, valid_loader, device)
          metrics_eval.append(eval_metric)

          # Save best model
          if eval_metric['loss'] < best_loss_eval:
              torch.save(model.state_dict(), save_model + f'/{model_name}.pt')

          times.append(time.time() - epoch_start_time)
          # Print loss, acc end epoch
          print("-" * 80)
          print(
              "| End of epoch {:3d} | Time: {:5.2f}s | Train Loss {:8.3f} | Valid Loss {:5.3f}"
              .format(
                  epoch, time.time() - epoch_start_time, train_loss, eval_metric["loss"]
              )
          )
          print("-" * 80)

      # Load best model
      model.load_state_dict(torch.load(save_model + f'/{model_name}.pt'))
      model.eval()
      metrics = {
          'train_loss': train_losses,
          'metric_eval': metrics_eval,
          'time': times
      }
      return model, metrics

## Initialize training loop

In [9]:
from transformers import get_scheduler

save_model = "/content/drive/MyDrive/NLU_NCKH/notebook/Checkpoints/Hotel_BERT"
EPOCHS = 15
num_labels = len(df_train.columns[1:])
model = phoBERTClass(MODEL_NAME, num_labels)

def loss_fn(outputs, targets):
    return torch.nn.BCELoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

num_training_steps = EPOCHS * len(training_loader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
best_model, metrics = train(
    model, "hotel_BERT", save_model, optimizer, loss_fn, training_loader,
    validating_loader, EPOCHS, device, num_training_steps
)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Training:   0%|          | 0/2700 [00:00<?, ?it/s]

Training 1:   0%|          | 0/180 [00:00<?, ?it/s]

| Epoch   1 | Index:    0 | Train Loss    0.558
| Epoch   1 | Index:  100 | Train Loss    0.094
| Epoch   1 | Index:  179 | Train Loss    0.073


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
| End of epoch  12 | Time: 192.21s | Train Loss    0.020 | Valid Loss 0.038
--------------------------------------------------------------------------------


Training 13:   0%|          | 0/180 [00:00<?, ?it/s]

| Epoch  13 | Index:    0 | Train Loss    0.021


KeyboardInterrupt: 

# Inference

## Load model

In [10]:
ckpt_path = "/content/drive/MyDrive/NLU_NCKH/notebook/Checkpoints/Hotel_BERT/hotel_BERT.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
num_labels = len(df_train.columns[1:])
best_model = phoBERTClass(MODEL_NAME, num_labels)

best_model.load_state_dict(torch.load(ckpt_path))
best_model.to(device)

phoBERTClass(
  (pretrained_bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm)

## Predict

In [11]:
from tqdm import tqdm
def predict(model, testing_loader, device):
  predictions = []
  with torch.no_grad():
      for _, data in tqdm(enumerate(testing_loader, 0), desc="Predicting", total=len(testing_loader)):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

          outputs = model(ids, mask, token_type_ids)
          predictions.extend(outputs.reshape(-1, num_labels, 4).argmax(axis=-1).cpu().detach().numpy())

      return np.array(predictions)

In [28]:
from sklearn.preprocessing import OneHotEncoder

def print_acsa_pred(replacements, categories, y_pred, y_true):
    sentiments = map(lambda x: replacements[x], y_pred)
    y_true = map(lambda x: replacements[x], y_true)
    predicted = []
    actual = []
    for category, sentiment, y in zip(categories, sentiments, y_true):
        if sentiment: predicted.append(f'{category}, {sentiment}')
        if y: actual.append(f'{category}, {y}')

    print("Predicted: ", ' | '.join(predicted))
    print("Ground Truth: ", ' | '.join(actual))

def label_one_hot(labels):
    len = labels.shape[0]
    labels = labels.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(categories=[[0, 1, 2, 3]], dtype='uint8', sparse=False)
    return encoder.fit_transform(labels).reshape(len, -1, 4)

aspects = df_test.columns[1:]

y_true = df_test[aspects].values
y_pred = predict(best_model, testing_loader, device)


Predicting: 100%|██████████| 102/102 [00:21<00:00,  4.82it/s]


In [26]:
y_pred[:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [27]:
y_true[:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [29]:
replacements = {0: None, 1: 'positive', 2: 'negative', 3: 'neutral'}

for i in range(100):
  print("-"*100)
  print('Example:', df_test['review'][i])
  print_acsa_pred(replacements, aspects, y_pred[i], y_true[i])

----------------------------------------------------------------------------------------------------
Example: mọi người đều nói_chuyện lịch_sự
Predicted:  SERVICE#GENERAL, neutral
Ground Truth:  SERVICE#GENERAL, neutral
----------------------------------------------------------------------------------------------------
Example: khách_sạn ttc hotel premium ngọc_lan bố_trí 2 phòng_không ở sát nhau nhưng cùng 1 tầng
Predicted:  
Ground Truth:  ROOMS#DESIGN&FEATURES, negative
----------------------------------------------------------------------------------------------------
Example: trong phòng vẫn đảm_bảo được sự sạch_sẽ và tiện_nghi
Predicted:  ROOMS#CLEANLINESS, neutral | ROOM_AMENITIES#GENERAL, neutral
Ground Truth:  ROOMS#CLEANLINESS, neutral | ROOMS#DESIGN&FEATURES, neutral
----------------------------------------------------------------------------------------------------
Example: chọn_lựa khách_sạn ttc hotel premium ngọc_lan để nghỉ_ngơi là một quyết_định đúng_đắn
Predicted:  HOTE

## Aspect Detection

In [30]:
from sklearn.metrics import classification_report

aspect_test = []
aspect_pred = []
y_test_argmax = df_test[aspects].to_numpy()

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_test.append(bool(col_test) * aspects[index])
        aspect_pred.append(bool(col_pred) * aspects[index])

aspect_report = classification_report(aspect_test, aspect_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_test, aspect_pred, digits=4, zero_division=1))

                                precision    recall  f1-score   support

                                   0.9868    0.9932    0.9900     65760
        FACILITIES#CLEANLINESS     1.0000    0.7500    0.8571        16
            FACILITIES#COMFORT     1.0000    0.0000    0.0000         6
    FACILITIES#DESIGN&FEATURES     1.0000    0.1765    0.3000        17
            FACILITIES#GENERAL     0.7059    0.2667    0.3871        45
      FACILITIES#MISCELLANEOUS     1.0000    0.0000    0.0000        11
             FACILITIES#PRICES     1.0000    0.0000    0.0000         6
            FACILITIES#QUALITY     0.7500    0.4091    0.5294        22
     FOOD&DRINKS#MISCELLANEOUS     1.0000    0.0000    0.0000         9
            FOOD&DRINKS#PRICES     1.0000    0.1250    0.2222         8
           FOOD&DRINKS#QUALITY     0.8370    0.7549    0.7938       102
     FOOD&DRINKS#STYLE&OPTIONS     0.9403    0.8514    0.8936        74
             HOTEL#CLEANLINESS     0.7895    0.5769    0.6667  

## Polarity Detection

In [31]:
y_test_flat = y_test_argmax.flatten()
y_pred_flat = y_pred.flatten()
target_names = list(map(str, replacements.values()))

polarity_report = classification_report(y_test_flat, y_pred_flat, digits=4, output_dict=True)
print(classification_report(y_test_flat, y_pred_flat, target_names=target_names, digits=4))

              precision    recall  f1-score   support

        None     0.9868    0.9932    0.9900     65760
    positive     0.7056    0.4117    0.5200       617
    negative     0.5000    0.2431    0.3271       181
     neutral     0.7912    0.7665    0.7786      2462

    accuracy                         0.9780     69020
   macro avg     0.7459    0.6036    0.6539     69020
weighted avg     0.9761    0.9780    0.9765     69020



## Aspect + Polarity

In [32]:
aspect_polarity_test = []
aspect_polarity_pred = []

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_polarity_test.append(f'{aspects[index]},{replacements[col_test]}')
        aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')

aspect_polarity_report = classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1))

                                         precision    recall  f1-score   support

            FACILITIES#CLEANLINESS,None     0.9980    1.0000    0.9990      2014
         FACILITIES#CLEANLINESS,neutral     1.0000    0.0000    0.0000         5
        FACILITIES#CLEANLINESS,positive     0.7500    0.8182    0.7826        11
                FACILITIES#COMFORT,None     0.9970    1.0000    0.9985      2024
             FACILITIES#COMFORT,neutral     1.0000    0.0000    0.0000         2
            FACILITIES#COMFORT,positive     1.0000    0.0000    0.0000         4
        FACILITIES#DESIGN&FEATURES,None     0.9931    1.0000    0.9965      2013
     FACILITIES#DESIGN&FEATURES,neutral     1.0000    0.0000    0.0000         5
    FACILITIES#DESIGN&FEATURES,positive     1.0000    0.2500    0.4000        12
                FACILITIES#GENERAL,None     0.9836    0.9975    0.9905      1985
            FACILITIES#GENERAL,negative     1.0000    0.0000    0.0000         3
             FACILITIES#GEN

# Summary

In [17]:
# Create dict score
aspect_dict = aspect_report['macro avg']
aspect_dict['accuracy'] = aspect_report['accuracy']

polarity_dict  = polarity_report['macro avg']
polarity_dict['accuracy'] = polarity_report['accuracy']

aspect_polarity_dict = aspect_polarity_report['macro avg']
aspect_polarity_dict['accuracy'] = aspect_polarity_report['accuracy']

# Create dataframe for dict_score
df_report = pd.DataFrame.from_dict([aspect_dict, polarity_dict, aspect_polarity_dict])
df_report.index = ['Aspect Detection', 'Polarity Detection', 'Aspect + Polarity']
df_report.drop('support', axis=1)

,precision,recall,f1-score,accuracy
Aspect Detection,0.856097,0.452596,0.501896,0.981049
Polarity Detection,0.746412,0.604123,0.654266,0.978064
Aspect + Polarity,0.882207,0.461280,0.476589,0.978064
